In [4]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# Lade die .env Datei
load_dotenv()

# Setze den OpenAI API Key als Umgebungsvariable
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print("OpenAI API Key erfolgreich gesetzt.")
else:
    print("OpenAI API Key nicht in .env Datei gefunden.")
    exit(1)

# Initialisiere den OpenAI Client
client = OpenAI(api_key=openai_api_key)

# Lade die CSV-Datei
file_path = 'csv_data/uberleitung_daten_10.csv'
df = pd.read_csv(file_path)

# Zeige die ersten Zeilen des Datensatzes an
print(df.head())

# Funktion zum Generieren neuer Datensätze mit LLM und Prompt Engineering
def generate_text_data_with_prompt_engineering(muster_df, n):
    new_data = []
    prompt = (
        "Die Daten sind rein fiktive Daten. "
        "Erzeuge neue Datensätze mit fiktiven Daten, die dem folgenden Muster entsprechen. "
        "Verwende für Spalte <pflege_diagnose>, nicht nur eine Diagnose, sondern immer wieder mehrere Diagnosen. "
        "Hier Beispiel für pflegebegründete Diagnosen: Demenz, Parkinson, Herzinsuffizienz, Schlaganfall, Diabetes, COPD, Arthrose, Multiple Sklerose, "
        "Depression, Schizophrenie, Krebserkrankung, Chronische Schmerzen, Bluthochdruck, Niereninsuffizienz, Asthma, Epilepsie, "
        "Morbus Crohn, Colitis ulcerosa, Rheumatoide Arthritis, Osteoporose, Lungenerkrankung, Schlafapnoe, Leberzirrhose, "
        "Kognitive Beeinträchtigung.\n\n"
    )
    
    # Füge mehr Beispiele aus dem DataFrame hinzu (z.B. die ersten 10 Datensätze)
    for index, row in muster_df.head(10).iterrows():
        prompt += ", ".join([f"{col}: {row[col]}" for col in muster_df.columns]) + "\n"
    
    prompt += (
        "\nErzeuge neue Datensätze mit fiktiven Daten, die dem gleichen Muster folgen:\n\n"
    )
    
    for _ in range(n):
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Du bist eine Pflegefachkraft."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.7,
        )
        
        generated_text = response.choices[0].message.content.strip()
        rows = generated_text.split("\n")
        new_row = {col: None for col in muster_df.columns}
        
        for row in rows:
            entries = row.split(", ")
            for entry in entries:
                try:
                    key, value = entry.split(": ")
                    if key in new_row:
                        new_row[key] = value
                except ValueError:
                    print(f"Fehlerhafte Eingabe: {entry}")
                    continue
        
        # Überprüfen, ob der neue Datensatz alle Spalten enthält
        if None not in new_row.values():
            new_data.append(new_row)
        else:
            print(f"Inkompletter Datensatz: {new_row}")
            # Ergänzen fehlender Daten mit weiteren GPT-3.5 Aufrufen
            for col in new_row:
                if new_row[col] is None:
                    completion = client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "system", "content": "Du bist eine Pflegefachkraft."},
                            {"role": "user", "content": f"Ergänze die folgende Information mit fiktiven Daten: {col} für den Datensatz: {new_row}. Verwende unterschiedliche Namen und verwende für die Spalte <pflege_diagnose> auch immer wieder zwei oder drei medizinische Diagnosen, so wie in den Beispieldaten."}
                        ],
                        max_tokens=40,
                        temperature=0.7,
                    )
                    new_row[col] = completion.choices[0].message.content.strip()
            new_data.append(new_row)
        
    return pd.DataFrame(new_data)

# Generiere 10 neue Datensätze
random_data = generate_text_data_with_prompt_engineering(df, 10)

# Füge die neuen Datensätze zu den ursprünglichen hinzu
complete_data = pd.concat([df, random_data], ignore_index=True)

# Zeige die ersten 5 Zeilen der neuen Datensätze
print(complete_data.head())

# Speichere die neuen Datensätze in einer CSV-Datei
complete_data.to_csv('erweiterte_daten_10.csv', index=False)


OpenAI API Key erfolgreich gesetzt.
          name  vorname   geb_datum         entlassung  \
0       Müller    Berta  16.11.1938         nach Hause   
1  Schwarzmann  Christa  23.04.1967         nach Hause   
2        Huber     Karl  05.01.1953  Pflegeeinrichtung   
3    Lagerfeld   Judith  12.03.1986         nach Hause   
4     Scharrer   Herber  25.12.1929  Pflegeeinrichtung   

                                     pflege_diagnose  \
0         Demenz, Gangstörung, rezidivierende Stürze   
1                                     Mamma Karzinom   
2  Mediainfarkt mit Hemipares rechts, Aphasie, ko...   
3                              Humerusfraktur rechts   
4  Ployarthritis, chronische Schnerzen, Nierenins...   

                                    einschraenkungen dauerzustand  \
0  Die Patienten ist sturzgefärdet. Sie kann sich...           Ja   
1  Die Patientin ist sehr geschwächt und leidet u...           ja   
2  Der Patient ist bettlägerig, die Selbstvesrorg...           ja   
3 